# Restricted Boltzman machine

In [1]:
import numpy as np

## Data

In [10]:
patterns = np.zeros((14,3,3))

patterns[1] = np.array([[1,0,0],[1,0,0],[1,0,0]])
patterns[2] = np.array([[0,1,0],[0,1,0],[0,1,0]])
patterns[3] = np.array([[0,0,1],[0,0,1],[0,0,1]])
patterns[4] = np.array([[1,1,0],[1,1,0],[1,1,0]])
patterns[5] = np.array([[0,1,1],[0,1,1],[0,1,1]])
patterns[6] = np.array([[1,0,1],[1,0,1],[1,0,1]])
patterns[7] = np.array([[1,1,1],[1,1,1],[1,1,1]])
patterns[8] = np.array([[1,1,1],[0,0,0],[0,0,0]])
patterns[9] = np.array([[0,0,0],[1,1,1],[0,0,0]])
patterns[10] = np.array([[0,0,0],[0,0,0],[1,1,1]])
patterns[11] = np.array([[1,1,1],[1,1,1],[0,0,0]])
patterns[12] = np.array([[0,0,0],[1,1,1],[1,1,1]])
patterns[13] = np.array([[1,1,1],[1,1,1],[0,0,0]])

patterns

array([[[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.]],

       [[0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.]],

       [[0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.]],

       [[1., 1., 0.],
        [1., 1., 0.],
        [1., 1., 0.]],

       [[0., 1., 1.],
        [0., 1., 1.],
        [0., 1., 1.]],

       [[1., 0., 1.],
        [1., 0., 1.],
        [1., 0., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [1., 1., 1.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.],
        [0., 0., 0.]]])